# Build the models and save them off 

In [3]:
#Import necessary packages
import psycopg2
import sys  
sys.path.append('..')

import numpy as np
import pandas as pd

from config import REDSHIFT_CONFIG
from src.features import *
from src.utils import *
from src.validation import *

reload(sys)
sys.setdefaultencoding('utf8')

from src.pitcher_class import *
from src.exploration import *

In [4]:
# Establish a connection to the redshift database
conn = create_rs_conn(config=REDSHIFT_CONFIG)
cur = conn.cursor()

In [5]:
# Create a list of the columns that we're interested in using as features
cols_of_interest = ([u'b', u's', u'on_1b', u'on_2b', u'on_3b', u'o',
                     u'home_wins',u'home_loss', u'away_wins', u'away_loss',
                     u'stand_L', u'Not_Fastball_pb_prior', u'Not_Fastball_pbs_prior', 
                     u'Fastball_pb_prior', u'Fastball_pbs_prior', u'Not_Fastball_pc_prior', 
                     u'Not_Fastball_pcs_prior', u'Fastball_pc_prior', u'Fastball_pcs_prior', 
                     u'Not_Fastball_pg_prior', u'Not_Fastball_pgs_prior', 
                     u'Fastball_pg_prior', u'Fastball_pgs_prior', u'last_pitch_type_Fastball', 
                     u'last_pitch_type_Not_Fastball',u'last_pitch_type_not_available', 
                     u'second_last_pitch_type_Fastball',u'second_last_pitch_type_Not_Fastball',
                     u'second_last_pitch_type_not_available', u'third_last_pitch_type_Fastball',
                     u'third_last_pitch_type_Not_Fastball', u'third_last_pitch_type_not_available', 
                     u'prev_pitches_mean_start_speed', u'prev_pitches_mean_end_speed',
                     u'prev_pitches_mean_break_y', u'prev_pitches_mean_break_angle',
                     u'prev_pitches_mean_break_length', u'ingame_pitch_count', u'cur_season', u'season_pitch_count'])

In [6]:
cur.execute('''SELECT pitcher,
                    p_first_name,
                    p_last_name,
                    COUNT(*) as tot_pitch_count,
                    MAX(date) as maximum_date,
                    AVG(CASE WHEN pitch_type IN ('FA', 'FF', 'FT', 'FC', 'FS', 'SI', 'SF') THEN 1.0
                        ELSE 0.0 END) as fastball_perc
                FROM 
                    all_pitch_data
                GROUP BY pitcher, p_first_name, p_last_name
                HAVING count(*) >= %d AND 
                MAX(date) > '%s' AND
                AVG(CASE WHEN pitch_type IN ('FA', 'FF', 'FT', 'FC', 'FS', 'SI', 'SF') THEN 1.0
                        ELSE 0.0 END) BETWEEN 0.4 AND 0.6
                ORDER BY pitcher''' % (3000, '2015-01-01'))

In [7]:
rows = cur.fetchall()
header = [colnames[0] for colnames in cur.description]
pitcher_df = pd.DataFrame(rows)
pitcher_df.columns = header
pitcher_df

,pitcher,p_first_name,p_last_name,tot_pitch_count,maximum_date,fastball_perc
0,115629,LaTroy,Hawkins,5918,2015-07-20,0.6
1,136600,Bruce,Chen,12153,2015-05-15,0.4
2,150274,Joe,Nathan,6239,2015-04-06,0.5
3,150302,Jason,Marquis,13759,2015-05-25,0.5
4,150359,A.J.,Burnett,25345,2015-07-20,0.6
5,276351,Jason,Grilli,6146,2015-07-11,0.6
6,276542,Joaquin,Benoit,6843,2015-07-20,0.5
7,279571,Matt,Belisle,7676,2015-06-25,0.6
8,279824,Mark,Buehrle,24295,2015-07-21,0.6
9,282332,CC,Sabathia,24873,2015-07-19,0.5


In [35]:
def randomly_sample_pitchers3(cursor, num_pitchers = 5, min_pitch_count = 600, min_date = '2015-01-01', seed_num = None):
    '''Takes a random sample of pitchers from the db represented by "cursor" and returns a Pandas DF with
    the specified number ofpitchers who have thrown at least "min_pitch_count" pitches
    Input:
        cursor: DB handle
        num_pitchers: The number of pitchers whose data you want returned
        min_pitch_count: Minimum number of pitches a pitcher must have thrown in order to be considered in the 
            random sampling
        seed_num: If you want to be able to replicated the results, set a seed
    Output: Pandas DF containing pitcher ids, the number of pitches they've thrown and their max pitch date'''
    cur = cursor
    
    #Get all pitchers meeting the min pitches criterion
    get_pitchers_query = '''SELECT pitcher,
                    p_first_name,
                    p_last_name,
                    COUNT(*) as tot_pitch_count,
                    MAX(date) as maximum_date,
                    AVG(CASE WHEN pitch_type IN ('FA', 'FF', 'FT', 'FC', 'FS', 'SI', 'SF') THEN 1.00
                        ELSE 0.00 END) as fastball_perc,
                    SUM(CASE WHEN pitch_type IN ('FA', 'FF', 'FT', 'FC', 'FS', 'SI', 'SF') THEN 1.0
                        ELSE 0.0 END) as num_fastballs,
                    SUM(CASE WHEN pitch_type NOT IN ('FA', 'FF', 'FT', 'FC', 'FS', 'SI', 'SF') THEN 1.0
                        ELSE 0.0 END) as num_not_fastballs
                FROM 
                    all_pitch_data
                GROUP BY pitcher, p_first_name, p_last_name
                HAVING count(*) >= %d AND 
                MAX(date) > '%s' AND
                AVG(CASE WHEN pitch_type IN ('FA', 'FF', 'FT', 'FC', 'FS', 'SI', 'SF') THEN 1.00
                        ELSE 0.00 END) BETWEEN 0.40 AND 0.60
                ORDER BY pitcher''' % (min_pitch_count, min_date)
    cur.execute(get_pitchers_query)
    
    #Get all the pitcher ids and sample from them
    if seed_num is not None:
        seed(seed_num)
    
    rows = cur.fetchall()
    header = [colnames[0] for colnames in cur.description]
    pitcher_df = pd.DataFrame(rows)
    pitcher_df.columns = header
    
    pitcher_id_sample = sample(pitcher_df['pitcher'].values, num_pitchers)
    
    pitcher_df = pitcher_df[pitcher_df['pitcher'].isin(pitcher_id_sample)]
    
    return pitcher_df

In [33]:
pitchers = randomly_sample_pitchers3(cur, min_pitch_count=2000, seed_num=35)

In [34]:
pitchers

,pitcher,p_first_name,p_last_name,tot_pitch_count,maximum_date,fastball_perc,num_fastballs,num_not_fastballs
0,136600,Bruce,Chen,12153,2015-05-15,0.44,5417.0,6732.0
1,150274,Joe,Nathan,6239,2015-04-06,0.56,3540.0,2665.0
2,150302,Jason,Marquis,13759,2015-05-25,0.57,7903.0,5796.0
3,276542,Joaquin,Benoit,6843,2015-07-20,0.55,3793.0,3020.0
4,282332,CC,Sabathia,24873,2015-07-19,0.56,14161.0,10339.0
5,285064,Ryan,Vogelsong,13055,2015-07-20,0.52,6864.0,6185.0
6,346798,Kyle,Lohse,20953,2015-07-12,0.52,11021.0,9809.0
7,407890,Colby,Lewis,13420,2015-07-18,0.58,7834.0,5558.0
8,408061,Francisco,Rodriguez,8227,2015-07-18,0.55,4528.0,3626.0
9,425386,Jeremy,Guthrie,24366,2015-07-18,0.57,14059.0,9964.0


In [27]:
pitcher_list = pitchers['pitcher'].values

### Get pitcher ids for the guys pitching tomorrow night 

In [28]:
cur.execute('''SELECT 
                    count(*) as num_pitches,
                    MAX(date) as latest_pitch_date,
                    p_first_name,
                    p_last_name,
                    pitcher
                FROM all_pitch_data
                WHERE p_first_name IN ('Williams', 'Colin', 'Cole', 'Scott', 'Matt', 'Carlos', 'Andrew') AND
                p_last_name IN ('Perez', 'Rea', 'Hamels', 'Kazmir', 'Garza', 'Rodon', 'Heaney')
                GROUP BY p_first_name, p_last_name, pitcher;''')
rows = cur.fetchall()
header = [colnames[0] for colnames in cur.description]
pitcher_df = pd.DataFrame(rows)
pitcher_df.columns = header
pitcher_df

,num_pitches,latest_pitch_date,p_first_name,p_last_name,pitcher
0,21671,2015-07-21,Matt,Garza,490063
1,26067,2015-07-19,Cole,Hamels,430935
2,16327,2015-07-18,Scott,Kazmir,431148
3,940,2015-07-20,Andrew,Heaney,571760
4,1331,2015-07-21,Carlos,Rodon,607074
5,804,2015-06-26,Williams,Perez,554234


In [29]:
pitcher_list = np.append(pitcher_list, pitcher_df['pitcher'].values)
pitcher_list

array([448306, 476454, 518716, 543359, 573185, 490063, 430935, 431148,
       571760, 607074, 554234], dtype=int64)

In [ ]:
for pitcher in pitcher_list:
    

In [68]:
pitcher = pitcher_list[0]

In [30]:
accuracies = {}
erros = []

for pitcher in pitcher_list[:2]:
    
    try:
        print 'starting pitcher:', pitcher
        test = Pitcher(pitcher_id = pitcher, redshift_cursor = cur)
        #test dates with Jason's code
        #test.subset_data_by_date(max_date = '2014-01-01')
        print test.pitch_type_by_year()
        test.split_test_train()
        print 'baseline acc:', test.baseline_accuracy
        test.run_classifiers()
        accuracies[pitcher] = {}
        accuracies[pitcher]['improvement'] = test.acc_over_most_common
        accuracies[pitcher]['rel_improvement'] = test.acc_over_most_common / (1 - test.baseline_accuracy)
        accuracies[pitcher]['baseline'] = test.baseline_accuracy
        print "\n"
        
    except:
        print "something went wrong with", pitcher, "\n"
        errors.append(pitcher)

In [31]:
accuracies

{448306: {'baseline': 0.605,
  'improvement': 0.058742690058479541,
  'rel_improvement': 0.14871567103412542},
 476454: {'baseline': 0.502,
  'improvement': 0.10274308300395252,
  'rel_improvement': 0.20631141165452313}}

To do:
* method in pitcher class to output data in the way Jason wants it
* loop to go through all pitchers and build models and save them
* Find pitcher ids of Cole Hamels, Scott Kazmir, and Matt Garza
* Function to train models on all of the data (not just the train)

In [ ]:
tester = 